In [253]:
# dependencies
from langchain_openai import ChatOpenAI ## for openAI
from langchain_google_genai import ChatGoogleGenerativeAI ## for gemini
from langchain_google_vertexai import VertexAI, ChatVertexAI, VertexAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.tools import tool
from pinecone.grpc import PineconeGRPC as pinecone_grpc
from langchain_core.output_parsers import JsonOutputParser
import os
from dotenv import load_dotenv
import vertexai



In [537]:
load_dotenv() # load environment variables
vertexai.init(project=os.getenv("PROJECT_ID"), location=os.getenv("PROJECT_LOCATION"))

In [757]:
llm = ChatVertexAI(model_name="gemini-1.5-flash-001",temperature=0.3)
# llm = ChatOpenAI(model="gpt-4o")
llm.invoke("In no more than 10 words. Who are you and do you support function calling?").content

I0000 00:00:1722207704.032193 1142885 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


"I'm a large language model, and yes. \n"

In [758]:
class PineconeClass:
  try:

    def __init__(self,user_id:str) -> None:
      self.user_id = user_id
      self.pc = pinecone_grpc(api_key=os.getenv("PINECONE_API_KEY"))
      ## pinecone db index
      self.index = self.pc.Index(os.getenv("PINECONE_INDEX_NAME"))
    
    def insert_vectors(self,images_vector:list[dict])->int:
      upsert_response = self.index.upsert(
        vectors=images_vector,
        namespace=self.user_id,
      )
      return upsert_response.upserted_count ## returns the total number of vectors inserted/updated

    def fetch_similar_vectors(self, vector_list:list[float|int], top_k:int=3,include_values:bool=False,filter:dict={},include_metadata:bool=False):
      similar_vectors = self.index.query(
        namespace=self.user_id,
        vector=vector_list,
        filter=filter,
        top_k=top_k,
        include_values=include_values,
        include_metadata=include_metadata
      )
      return similar_vectors
    
  except Exception as e:
    raise Exception(f"Error in PineconeClass: {str(e)}")
  


In [759]:
# generate text embeddings
def generate_text_embeddings(user_prompt:str):
  try:
    embeddings_model= VertexAIEmbeddings(model_name="multimodalembedding")
    embeddings = embeddings_model.embed_query(user_prompt) # Generate embeddings
    return embeddings
  except Exception as e:
    print(f"Error in generate_text_embeddings function: {str(e)}")

In [760]:
embeddings = generate_text_embeddings("what to wear for a cruise party today evening in toronto?")
print(embeddings[:5])
print(len(embeddings))

I0000 00:00:1722207705.477168 1142885 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1722207705.692868 1403648 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1722207705.694753 1403648 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


[-0.035197977, -0.0411336273, -0.00809537619, 0.0147745684, -0.00912695844]
1408


In [761]:
# agent tools
lowerwear_collection = []
upperwear_collection = []
@tool
def retrieve_upperwear(user_id:str,user_prompt:str)->list[dict]:
  """fetches the top 4 upperwear vectors closest to the vector of the prompt_vector from the pinecone db vector store"""
  try:
    pinecone_instance = PineconeClass(user_id=user_id)
    embeddings = generate_text_embeddings(user_prompt=user_prompt)
    upperwear_results = pinecone_instance.fetch_similar_vectors(vector_list=embeddings,top_k=4,filter={"tag":"upperwear"},include_metadata=True)["matches"]
    upperwear_items = [{"type":"image_url","image_url":result["metadata"]["url"]} for result in upperwear_results]
    global upperwear_collection
    upperwear_collection = upperwear_items
    return upperwear_items
  except Exception as e:
    print(f"Error in get_similar_clothing_items tool: {str(e)}")

@tool
def retrieve_lowerwear(user_id:str,user_prompt:str)->list[dict]:
  """fetches the top 4 lowerwear vectors closest to the vector of the prompt_vector from the pinecone db vector store"""
  try:
    pinecone_instance = PineconeClass(user_id=user_id)
    embeddings = generate_text_embeddings(user_prompt=user_prompt)
    lowerwear_results = pinecone_instance.fetch_similar_vectors(vector_list=embeddings,top_k=4,filter={"tag":"lowerwear"},include_metadata=True)["matches"]
    lowerwear_items = [{"type":"image_url","image_url":result["metadata"]["url"]} for result in lowerwear_results]
    global lowerwear_collection
    lowerwear_collection = lowerwear_items
    return lowerwear_items
  except Exception as e:
    print(f"Error in get_similar_clothing_items tool: {str(e)}")


In [762]:
prompt = ChatPromptTemplate.from_messages([
  ("system", "You are a powerful AI bot whose job is to fetch the data. When the user asks for a recommendation, you will first use the tools 'retrieve_upperwear' and 'retrieve_lowerwear' to fetch the top 4 upperwear and lowerwear vectors closest. End the chain my mentioning the clothing item links you fetched and end it with a positive note."),
  ("human","user_prompt is {user_prompt} and the user_id is {user_id}"),
  ("placeholder","{agent_scratchpad}")
  
])
tools=[retrieve_upperwear,retrieve_lowerwear]

In [763]:
agent = create_tool_calling_agent(llm=llm,tools=tools,prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True) 

In [764]:
resp = agent_executor.invoke({"user_prompt":"recommend me something to wear at a gym?","user_id":"JKVDl1ErPjaj3TPRNuBUsN3W9xS2"}) 
print(resp)



> Entering new AgentExecutor chain...

Invoking: `retrieve_upperwear` with `{'user_id': 'JKVDl1ErPjaj3TPRNuBUsN3W9xS2', 'user_prompt': 'recommend me something to wear at a gym?'}`




I0000 00:00:1722207718.054365 1142885 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1722207718.269313 1403746 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1722207718.269937 1403746 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


[{'type': 'image_url', 'image_url': 'https://firebasestorage.googleapis.com/v0/b/perobeai-430021.appspot.com/o/JKVDl1ErPjaj3TPRNuBUsN3W9xS2%2FshirtPlain3641cf606-e83c-4aea-b2a6-a852cfacad0f.png?alt=media&token=3f6a86da-fd41-48f9-b047-487c56954535'}, {'type': 'image_url', 'image_url': 'https://firebasestorage.googleapis.com/v0/b/perobeai-430021.appspot.com/o/JKVDl1ErPjaj3TPRNuBUsN3W9xS2%2F9shirtFullSleeves9e5e142b-0173-49c9-b54c-e75e279465b5.png?alt=media&token=c4f213ff-e4d4-41be-a176-b2d3ad62299c'}, {'type': 'image_url', 'image_url': 'https://firebasestorage.googleapis.com/v0/b/perobeai-430021.appspot.com/o/JKVDl1ErPjaj3TPRNuBUsN3W9xS2%2F6tshirtFormal15721fec-107d-49c5-805e-a32ec8713139.png?alt=media&token=9565ba77-f8bf-4b24-be09-278021937c0f'}, {'type': 'image_url', 'image_url': 'https://firebasestorage.googleapis.com/v0/b/perobeai-430021.appspot.com/o/JKVDl1ErPjaj3TPRNuBUsN3W9xS2%2FpoloShirts3fe99a52-d510-40e8-8bff-6a6b39d4cecf.png?alt=media&token=9416921a-ef57-43c3-ac08-f542567a3658

I0000 00:00:1722207719.998329 1142885 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1722207720.213660 1403758 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1722207720.216656 1403758 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


[{'type': 'image_url', 'image_url': 'https://firebasestorage.googleapis.com/v0/b/perobeai-430021.appspot.com/o/JKVDl1ErPjaj3TPRNuBUsN3W9xS2%2Fshorts0f1b8a67-5abe-4eb8-a0af-2e196cfe7e29.png?alt=media&token=33eb2b99-229f-4c46-b58d-b06032aaf497'}, {'type': 'image_url', 'image_url': 'https://firebasestorage.googleapis.com/v0/b/perobeai-430021.appspot.com/o/JKVDl1ErPjaj3TPRNuBUsN3W9xS2%2F1pants4f3912a4-14bf-4f46-a255-89677ac76b37.png?alt=media&token=7b8f89ff-7bd8-46d5-8b6a-eaa6c4292372'}, {'type': 'image_url', 'image_url': 'https://firebasestorage.googleapis.com/v0/b/perobeai-430021.appspot.com/o/JKVDl1ErPjaj3TPRNuBUsN3W9xS2%2Flowerbc5634c0-4cde-4262-afe0-d3e16c4586a2.png?alt=media&token=ab69c9c9-6d7d-4b23-b83b-0a3159fdc562'}, {'type': 'image_url', 'image_url': 'https://firebasestorage.googleapis.com/v0/b/perobeai-430021.appspot.com/o/JKVDl1ErPjaj3TPRNuBUsN3W9xS2%2F3jeans384bff4b-2d07-4f31-be25-9acb5e86a359.png?alt=media&token=a3cac849-6c88-41d0-bd17-10def087434a'}]Here are some recommendat

In [765]:
upperwear_collection

[{'type': 'image_url',
  'image_url': 'https://firebasestorage.googleapis.com/v0/b/perobeai-430021.appspot.com/o/JKVDl1ErPjaj3TPRNuBUsN3W9xS2%2FshirtPlain3641cf606-e83c-4aea-b2a6-a852cfacad0f.png?alt=media&token=3f6a86da-fd41-48f9-b047-487c56954535'},
 {'type': 'image_url',
  'image_url': 'https://firebasestorage.googleapis.com/v0/b/perobeai-430021.appspot.com/o/JKVDl1ErPjaj3TPRNuBUsN3W9xS2%2F9shirtFullSleeves9e5e142b-0173-49c9-b54c-e75e279465b5.png?alt=media&token=c4f213ff-e4d4-41be-a176-b2d3ad62299c'},
 {'type': 'image_url',
  'image_url': 'https://firebasestorage.googleapis.com/v0/b/perobeai-430021.appspot.com/o/JKVDl1ErPjaj3TPRNuBUsN3W9xS2%2F6tshirtFormal15721fec-107d-49c5-805e-a32ec8713139.png?alt=media&token=9565ba77-f8bf-4b24-be09-278021937c0f'},
 {'type': 'image_url',
  'image_url': 'https://firebasestorage.googleapis.com/v0/b/perobeai-430021.appspot.com/o/JKVDl1ErPjaj3TPRNuBUsN3W9xS2%2FpoloShirts3fe99a52-d510-40e8-8bff-6a6b39d4cecf.png?alt=media&token=9416921a-ef57-43c3-ac08-f

In [744]:
lowerwear_collection

[{'type': 'image_url',
  'image_url': 'https://firebasestorage.googleapis.com/v0/b/perobeai-430021.appspot.com/o/JKVDl1ErPjaj3TPRNuBUsN3W9xS2%2Fshorts0f1b8a67-5abe-4eb8-a0af-2e196cfe7e29.png?alt=media&token=33eb2b99-229f-4c46-b58d-b06032aaf497'},
 {'type': 'image_url',
  'image_url': 'https://firebasestorage.googleapis.com/v0/b/perobeai-430021.appspot.com/o/JKVDl1ErPjaj3TPRNuBUsN3W9xS2%2F1pants4f3912a4-14bf-4f46-a255-89677ac76b37.png?alt=media&token=7b8f89ff-7bd8-46d5-8b6a-eaa6c4292372'},
 {'type': 'image_url',
  'image_url': 'https://firebasestorage.googleapis.com/v0/b/perobeai-430021.appspot.com/o/JKVDl1ErPjaj3TPRNuBUsN3W9xS2%2Flowerbc5634c0-4cde-4262-afe0-d3e16c4586a2.png?alt=media&token=ab69c9c9-6d7d-4b23-b83b-0a3159fdc562'},
 {'type': 'image_url',
  'image_url': 'https://firebasestorage.googleapis.com/v0/b/perobeai-430021.appspot.com/o/JKVDl1ErPjaj3TPRNuBUsN3W9xS2%2F3jeans384bff4b-2d07-4f31-be25-9acb5e86a359.png?alt=media&token=a3cac849-6c88-41d0-bd17-10def087434a'}]